In [1]:
!pip install openpyxl

In [ ]:
### SQL for inital transformation
"""
with unpivoted as (
    select
        company,
        account,
        year,
        nullif(value, '-') as value  -- value is always string; converts '-' to NULL
    from {{ ref('stg_annual_financial_statements') }}
    unpivot (
        value for year in (`2020`, `2021`, `2022`, `2023`, `2024`)
    )
    where lower(company) != 'company'
      and lower(account) != 'account'
),

numeric_pivot as (
    select
        company,
        year,
        max(case when account = 'Intäkter' then cast(value as float64) end) as revenue,
        max(case when account = 'Bruttoresultat' then cast(value as float64) end) as gross_profit,
        max(case when account = 'Rörelseresultat' then cast(value as float64) end) as operating_profit,
        max(case when account = 'Resultat efter finansiella poster' then cast(value as float64) end) as profit_after_financial_items,
        max(case when account = 'Nettoresultat' then cast(value as float64) end) as net_profit,
        max(case when account = 'Vinst per aktie' then cast(value as float64) end) as earnings_per_share,
        max(case when account = 'Utdelning' then cast(value as float64) end) as dividend,
        max(case when account = 'Materiella anläggningstillgångar' then cast(value as float64) end) as tangible_fixed_assets,
        max(case when account = 'Immateriella anläggningstillgångar' then cast(value as float64) end) as intangible_fixed_assets,
        max(case when account = 'Finansiella anläggningstillgångar' then cast(value as float64) end) as financial_fixed_assets,
        max(case when account = 'Varulager' then cast(value as float64) end) as inventory,
        max(case when account = 'Kundfordringar' then cast(value as float64) end) as accounts_receivable,
        max(case when account = 'Kortfristiga placeringar' then cast(value as float64) end) as short_term_investments,
        max(case when account = 'Kassa och bank' then cast(value as float64) end) as cash_and_bank,
        max(case when account = 'Övriga omsättningstillgångar' then cast(value as float64) end) as other_current_assets,
        max(case when account = 'Totala tillgångar' then cast(value as float64) end) as total_assets,
        max(case when account = 'Eget kapital - Moderbolagets ägare' then cast(value as float64) end) as equity_parent_companys_shareholders,
        max(case when account = 'Minoritetsintressen' then cast(value as float64) end) as minority_interests,
        max(case when account = 'Långfristiga skulder' then cast(value as float64) end) as long_term_liabilities,
        max(case when account = 'Kortfristiga skulder' then cast(value as float64) end) as short_term_liabilities,
        max(case when account = 'Totalt eget kapital och skulder' then cast(value as float64) end) as total_equity_and_liabilities
    from unpivoted
    where account not in (
        'Valuta', 
        'Startdatum Resultaträkning', 
        'Slutdatum Resultaträkning', 
        'Startdatum Balansräkning', 
        'Slutdatum Balansräkning'
    )
    group by company, year
),

meta_pivot as (
    select
        company,
        year,
        max(case when account = 'Valuta' then value end) as currency,
        max(case when account = 'Startdatum Resultaträkning' then value end) as income_statement_start_date,
        max(case when account = 'Slutdatum Resultaträkning' then value end) as income_statement_end_date,
        max(case when account = 'Startdatum Balansräkning' then value end) as balance_sheet_start_date,
        max(case when account = 'Slutdatum Balansräkning' then value end) as balance_sheet_end_date
    from unpivoted
    where account in (
        'Valuta', 
        'Startdatum Resultaträkning', 
        'Slutdatum Resultaträkning', 
        'Startdatum Balansräkning', 
        'Slutdatum Balansräkning'
    )
    group by company, year
)

select
    n.company,
    DATE(CONCAT(n.year, '-01-01')) as fiscal_year,
    n.revenue,
    n.gross_profit,
    n.operating_profit,
    n.profit_after_financial_items,
    n.net_profit,
    n.earnings_per_share,
    n.dividend,
    n.tangible_fixed_assets,
    n.intangible_fixed_assets,
    n.financial_fixed_assets,
    n.inventory,
    n.accounts_receivable,
    n.short_term_investments,
    n.cash_and_bank,
    n.other_current_assets,
    n.total_assets,
    n.equity_parent_companys_shareholders,
    n.minority_interests,
    n.long_term_liabilities,
    n.short_term_liabilities,
    n.total_equity_and_liabilities,
    m.currency,
    m.income_statement_start_date,
    m.income_statement_end_date,
    m.balance_sheet_start_date,
    m.balance_sheet_end_date
from numeric_pivot n
left join meta_pivot m
   on n.company = m.company
  and n.year = m.year
where n.company is not null
"""

In [2]:
import pandas as pd

In [5]:
dfs = pd.read_excel('financial_statements.xlsx', sheet_name=None)

In [8]:
first = True
for sheet, data in dfs.items():
    if not sheet == 'MALL':
        data = data.rename(columns = {'Unnamed: 0':'Account'})
        data.columns = [str(col) for col in data.columns]
        str_cols = ['Valuta', 'Startdatum Resultaträkning', 'Slutdatum Resultaträkning', 'Startdatum Balansräkning', 'Slutdatum Balansräkning']
        data_str_cols = data[data.Account.isin(str_cols)]
        data_num_cols = data[~data.Account.isin(str_cols)]
        data_long = pd.melt(data_num_cols, id_vars = ['Account'], value_vars = years, var_name = 'Year', value_name = 'Value')
        data_long = data_long.dropna(subset=['Account', 'Year'])
        data_long.Value = data_long.Value.replace('-', '0')
        data_long.Value = data_long.Value.astype(float)
        data_wide = data_long.pivot(index='Year', columns='Account', values='Value').reset_index()
        
        # Put string values as columns to keep the 'Value' column numeric
        data_str_cols_melted = data_str_cols.melt(id_vars='Account', var_name='Year', value_name='value')
        data_str_cols_pivoted = data_str_cols_melted.pivot_table(index='Year', columns='Account', values='value', aggfunc='first').reset_index(drop = False)
        
        data_company = pd.merge(data_wide, data_str_cols_pivoted, on = 'Year')
        data_company['Company'] = sheet

        if first:
            data_all = data_company
            first = False
        else:
            data_all = pd.concat([data_all, data_company], axis = 0)
data_all = data_all.reset_index(drop = True)
data_all = data_all[['Company', 'Year', 'Intäkter', 'Bruttoresultat', 'Rörelseresultat', 'Resultat efter finansiella poster',
    'Nettoresultat', 'Vinst per aktie', 'Utdelning', 'Materiella anläggningstillgångar',
    'Immateriella anläggningstillgångar', 'Finansiella anläggningstillgångar', 'Varulager',
    'Kundfordringar', 'Kortfristiga placeringar', 'Kassa och bank', 'Övriga omsättningstillgångar',
    'Totala tillgångar', 'Eget kapital - Moderbolagets ägare', 'Minoritetsintressen',
    'Långfristiga skulder', 'Kortfristiga skulder', 'Totalt eget kapital och skulder', 'Valuta',
    'Startdatum Resultaträkning', 'Slutdatum Resultaträkning', 'Startdatum Balansräkning',
    'Slutdatum Balansräkning']]
data_all.columns = ['company', 'fiscal_year', 'revenue', 'gross_profit', 'operating_profit', 'profit_after_financial_items',
    'net_profit', 'earnings_per_share', 'dividend', 'tangible_fixed_assets',
    'intangible_fixed_assets', 'financial_fixed_assets', 'inventory',
    'accounts_receivable', 'short_term_investments', 'cash_and_bank', 'other_current_assets',
    'total_assets', 'equity_parent_companys_shareholders', 'minority_interests',
    'long_term_liabilities', 'short_term_liabilities', 'total_equity_and_liabilities', 'currency',
    'income_statement_start_date', 'income_statement_end_date', 'balance_sheet_start_date',
    'balance_sheet_end_date']

In [9]:
data_all.head()

,company,fiscal_year,revenue,gross_profit,operating_profit,profit_after_financial_items,net_profit,earnings_per_share,dividend,tangible_fixed_assets,...,equity_parent_companys_shareholders,minority_interests,long_term_liabilities,short_term_liabilities,total_equity_and_liabilities,currency,income_statement_start_date,income_statement_end_date,balance_sheet_start_date,balance_sheet_end_date
0,AAK,2020,27934.0,7726.0,2167.0,2040.0,1581.0,6.16,2.30,6310.0,...,9738.0,39.0,3902.0,8638.0,22278.0,MSEK,2020-01-01 00:00:00,2020-12-31 00:00:00,2020-01-01 00:00:00,2020-12-31 00:00:00
1,AAK,2021,35452.0,7941.0,2089.0,1987.0,1447.0,5.60,2.50,6315.0,...,11831.0,48.0,3180.0,12155.0,27166.0,MSEK,2021-01-01 00:00:00,2021-12-31 00:00:00,2021-01-01 00:00:00,2021-12-31 00:00:00
2,AAK,2022,50425.0,10648.0,2538.0,2350.0,1780.0,6.84,2.75,7629.0,...,15083.0,47.0,15083.0,4972.0,33990.0,MSEK,2022-01-01 00:00:00,2022-12-31 00:00:00,2022-01-01 00:00:00,2022-12-31 00:00:00
3,AAK,2023,46028.0,12607.0,4116.0,3824.0,2954.0,11.35,3.70,8253.0,...,17173.0,56.0,3741.0,9637.0,30551.0,MSEK,2023-01-01 00:00:00,2023-12-31 00:00:00,2023-01-01 00:00:00,2023-12-31 00:00:00
4,AAK,2024,45052.0,13709.0,4896.0,4660.0,3542.0,13.62,5.00,8594.0,...,20149.0,62.0,2533.0,11704.0,34386.0,MSEK,2024-01-01 00:00:00,2024-12-31 00:00:00,2024-01-01 00:00:00,2024-12-31 00:00:00


In [17]:
import pandas as pd
from pandas import DataFrame

def calc_average_revenue_growth(df: DataFrame) -> DataFrame:
    # Ensure correct types
    df['year'] = df['fiscal_year'].astype(int)
    df['revenue'] = df['revenue'].astype(float)

    # Sort values to ensure correct order for pct_change
    df = df.sort_values(["company", "year"])

    # Calculate yearly % revenue changes for each company
    df['revenue_pct_change'] = df.groupby('company')['revenue'].pct_change()

    # Compute the mean of % change per company (skip the first NaN)
    result = df.groupby('company')['revenue_pct_change'].mean().reset_index()
    result.rename(columns={'revenue_pct_change': 'revenue_growth'}, inplace=True)

    return result

In [18]:
average_revenue_growth = calc_average_revenue_growth(data_all)

In [25]:
import pandas as pd

def forecast_revenue_growth(avg_revenue_growth_df: DataFrame, forecast_years: int = 10, start_year: int = 2025) -> DataFrame:
    records = []
    for _, row in avg_revenue_growth_df.iterrows():
        company = row['company']
        avg_revenue_growth = row['revenue_growth']
        for year in range(1, forecast_years+1):
            if year == 1:
                fg = avg_revenue_growth
            elif year == 2:
                fg = 0.75 * avg_revenue_growth + 0.25 * 0.04
            elif year == 3:
                fg = 0.5 * avg_revenue_growth + 0.5 * 0.04
            elif year == 4:
                fg = 0.25 * avg_revenue_growth + 0.75 * 0.04
            else:
                fg = 0.04
            records.append({'company': company, 'year': year, 'fiscal_year': start_year + year - 1, 'forecasted_revenue_growth': fg})
    return pd.DataFrame(records)

In [26]:
forecasted_revenue_growth = forecast_revenue_growth(average_revenue_growth)

In [48]:
import pandas as pd

def forecast_revenues(historical_df: DataFrame, forecast_growth_df: DataFrame, latest_fiscal_year: int = 2024) -> DataFrame:
    """
    Forecast revenues using the last available (usually 2024) historical revenue and the forecasted growth rates.
    
    Parameters:
        historical_df (pd.DataFrame): Columns: 'company', 'fiscal_year', 'revenue'.
        forecast_growth_df (pd.DataFrame): Columns: 'company', 'fiscal_year', 'forecasted_revenue_growth'.
        latest_fiscal_year (int): The fiscal year to use as the base for forecasts.
    
    Returns:
        pd.DataFrame: Columns: 'company', 'fiscal_year', 'forecasted_revenue'.
    """
    
    historical_df.fiscal_year = historical_df.fiscal_year.astype(int) # Make sure correct type
    
    # Get the latest revenue for each company
    last_revenue = historical_df[historical_df['fiscal_year'] == latest_fiscal_year][['company', 'revenue']]
    last_revenue = last_revenue.rename(columns={'revenue': 'latest_revenue'})

    # Merge forecast_growth_df with last_revenue to get the starting value for each company
    forecast = forecast_growth_df.merge(last_revenue, on='company', how='left')
    print(forecast[['company', 'fiscal_year', 'latest_revenue', 'forecasted_revenue_growth']].head(10))
    forecast = forecast.sort_values(['company', 'fiscal_year'])

    # We'll build the output as a list of dicts to avoid in-place assignment problems
    records = []
    for company, group in forecast.groupby('company'):
        # Get the starting revenue for this company
        latest_revenue = group['latest_revenue'].iloc[0]
        revenue = latest_revenue
        for idx, row in group.iterrows():
            growth = row['forecasted_revenue_growth']
            revenue = revenue * (1 + growth)
            records.append({
                'company': company,
                'fiscal_year': row['fiscal_year'],
                'forecasted_revenue': revenue
            })

    forecast_out = pd.DataFrame(records)
    return forecast_out